In [105]:
import json
import random


In [116]:
class Sentiment:
    NEGATIVE = "NEGATIVE"
    POSITIVE = "POSITIVE"
    NEUTRAL = "NEUTRAL"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE
        
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews) 

    

## Get Data

In [117]:
file_name = "Books_big.json"

reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))
        
         
reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

## Prep Data for Training

In [118]:
from sklearn.model_selection import train_test_split

In [137]:
training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

train_container.evenly_distribute()
test_container.evenly_distribute()

len(cont.reviews)

872

In [138]:
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_x = test_container.get_text()
test_y = test_container.get_sentiment()

train_y.count(Sentiment.POSITIVE)
train_y.count(Sentiment.NEGATIVE)

436

## Bag of Words! 

In [161]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#tfid places greater weight on words that dont often appear in the review
#for example, "This book is great!" and "this book was bad"
#book appeared in both sentences and probably does not effect the sentiment negatively or positively

In [162]:
vectorizer = TfidfVectorizer()

#this next function has two jobs: 
#simultanously fits the training data and transforms it 
#vectorizer.fit(train_x)
#train_x_vectors = vectorizer.transform(train_x)
train_x_vectors = vectorizer.fit_transform(train_x) 

test_x_vectors = vectorizer.transform(test_x)


#the big matrix is pretty big because it accounts for all the words

## Classification 

### Linear SVM

In [163]:
from sklearn import svm

In [164]:
clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

test_x[0]

clf_svm.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

### Decision Tree

In [165]:
from sklearn.tree import DecisionTreeClassifier


In [166]:
clf_dec = DecisionTreeClassifier()

clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

### Naive Bayes

In [167]:
from sklearn.naive_bayes import GaussianNB

In [168]:
# clf_nb = GaussianNB()

# clf_nb.fit(train_x_vectors.toarray(), train_y)

# clf_nb.predict(test_x_vectors[0])

### Linear Regression

In [169]:
from sklearn.linear_model import LogisticRegression

In [170]:
clf_log = LogisticRegression()

clf_log.fit(train_x_vectors, train_y)

clf_log.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

## Evaluation 

In [171]:
#Mean Accuracy

print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_log.score(test_x_vectors, test_y))


0.8076923076923077
0.6394230769230769
0.8052884615384616


In [172]:
#F1 Scores
from sklearn.metrics import f1_score

In [173]:
print(f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))
print(f1_score(test_y, clf_dec.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))
print(f1_score(test_y, clf_log.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))

[0.80582524 0.80952381]
[0.63054187 0.64788732]
[0.80291971 0.80760095]


In [85]:
train_y.count(Sentiment.POSITIVE)
train_y.count(Sentiment.NEGATIVE)

436

In [157]:
test_set = ['yuck!', 'i loved it!', 'cannot wait for the next one', 'brilliant', 'this book sucked']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE'],
      dtype='<U8')

## Tuning our model

In [174]:
from sklearn.model_selection import GridSearchCV

In [177]:
parameters = {'kernel': ('linear', 'rbf'), 'C': (1, 4, 8, 16, 32)}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv = 5)
clf.fit(train_x_vectors, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [178]:
print(clf.score(test_x_vectors, test_y))

0.8197115384615384


## Saving Model

In [179]:
import pickle

In [182]:
with open('sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

 ## Load Model

In [183]:
with open('sentiment_classifier.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

In [185]:
loaded_clf.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')